# Predict Author Features

### Download and load Blog Authorship Corpus
- Link to dataset: https://www.kaggle.com/rtatman/blog-authorship-corpus/downloads/blog-authorship-corpus.zip/2at

### Dataset description
- You can find the dataset description here:
https://www.kaggle.com/rtatman/blog-authorship-corpus

In [138]:
df = pd.read_csv("Dataset - blogtext.csv")

In [139]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [140]:
print('Shape of the dataframe is', df.shape) # Shape of DataFrame

Shape of the dataframe is (681284, 7)


In [141]:
print('Total Number of Blogs:', df.shape[0])

Total Number of Blogs: 681284


In [142]:
df.isnull().any()  # Check if there are any null values

id        False
gender    False
age       False
topic     False
sign      False
date      False
text      False
dtype: bool

In [143]:
df.info()   # Information about the columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      681284 non-null  int64 
 1   gender  681284 non-null  object
 2   age     681284 non-null  int64 
 3   topic   681284 non-null  object
 4   sign    681284 non-null  object
 5   date    681284 non-null  object
 6   text    681284 non-null  object
dtypes: int64(2), object(5)
memory usage: 36.4+ MB


In [144]:
df.drop(['date'], axis=1, inplace=True)   # Drop the date column as it is not needed

In [145]:
print("Total number of bloggers:", df.groupby(by=["id", "gender", "age", "topic", "sign"]).first().shape[0])

Total number of bloggers: 19320


In [146]:
print('Average number of posts per blogger:', 
      int(df[['id','gender','age','topic','sign']].value_counts().mean()))

Average number of posts per blogger: 35


In [147]:
# Bringing all unique bloggers into one row after concatinating the text column
df['text'] = df.groupby(by=["id", "gender", "age", "topic", "sign"])['text'].transform(lambda x: ' '.join(x))
df.drop_duplicates(["id", "gender", "age", "topic", "sign"], inplace=True)

In [148]:
df['totalwords'] = [len(x.split()) for x in df['text'].tolist()]
print("Average number of words per blogger:", int(df['totalwords'].mean()))

Average number of words per blogger: 7080


## Consolidating analysis
Total Number of Blogs: 681284  
Null Values: NO  
Total number of bloggers: 19320  
Average number of posts per blogger: 35  
Average number of words per blogger: 7080

### Grouping the same ids together and concatinating the text into one single row is best way but taking too much time to process, hence, for reducing the computation and time, taking the first blog of each unique id (blogger)

#### Read the csv using pandas

In [60]:
import pandas as pd

df = pd.read_csv('Dataset - blogtext.csv')

#### Get the names of the columns

In [61]:
df.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

#### Have a look at some column values

In [62]:
df.head(5)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


#### Check if there is any null value, and get the total count.

In [63]:
df.isnull().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

### Cut the data (skip this step in final run)
Make your data short during development. So that overall process takes less time to execute and you are able to rectify all the errors fast, and check if your code is running smooth.
When evrything is sorted at last, load the entire data and run your code on that and skip this step.

In [64]:
df = df.head(3000)

## Preprocess text
Preprocess values of text column

- Remove unwanted characters
- Convert text to lowercase
- Remove unwanted spaces
- Remove stopwords

In [65]:
# Select only alphabets
import re
df.text = df.text.apply(lambda x: re.sub('[^A-Za-z]+', ' ', x))

# Convert text to lowercase
df.text = df.text.apply(lambda x: x.lower())

# Strip unwanted spaces
df.text = df.text.apply(lambda x: x.strip())

# Remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
df.text = df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaibh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Verify the preprocessing steps by looking over some values

In [66]:
df.text[6]

'somehow coca cola way summing things well early flagship jingle like buy world coke tune like teach world sing pretty much summed post woodstock era well add much sales catchy tune korea coke theme urllink stop thinking feel pretty much sums lot korea koreans look relaxed couple stopped thinking started feeling course high regard education math logic deep think many koreans really like work emotion anything else westerners seem sublimate moreso least display different way maybe scratch westerners koreans probably pretty similar context different anyways think losing korea repeat stop thinking feel stop thinking feel stop thinking feel everything alright'

### Merge the label columns

Merge all the label columns together, so that we have all the tags together for a particular sentence

In [67]:
df['labels'] = df.apply(lambda row: [row['gender'], str(row['age']), row['topic'], row['sign']], axis=1)

### Select only required columns from your dataframe

In [68]:
df = df[['text','labels']]

### Print final dataframe

In [69]:
df.head()

,text,labels
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


## Create training and testing data

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.text.values, df.labels.values, test_size=0.20, random_state=42)

## Vectorize the data

### Create Bag of Words
- Use CountVectorizer
- Transform the training and testing data

In [71]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

In [72]:
X_test_bow

<600x180703 sparse matrix of type '<class 'numpy.int64'>'
	with 41817 stored elements in Compressed Sparse Row format>

#### Have a look at some feature names

In [73]:
len(vectorizer.vocabulary_)

180703

In [74]:
vectorizer.get_feature_names()

['aa',
 'aa compared',
 'aa nice',
 'aaa',
 'aaa take',
 'aaa travel',
 'aaaaaah',
 'aaaaack',
 'aaaah',
 'aaaah wisdom',
 'aaaahh',
 'aaagh',
 'aaagh pero',
 'aaah',
 'aaah eat',
 'aaah hafta',
 'aaahhh',
 'aaahhh cryptic',
 'aal',
 'aal eliminate',
 'aal esseneth',
 'aal lost',
 'aal powerful',
 'aaldering',
 'aaldering urllink',
 'aamco',
 'aamco kind',
 'aand',
 'aand jim',
 'aarde',
 'aarde maak',
 'aargh',
 'aargh told',
 'aaron',
 'aaron brown',
 'aaron james',
 'aaron rowand',
 'aba',
 'aba foundation',
 'aba sessions',
 'aba sus',
 'aba therapy',
 'aba well',
 'aballaby',
 'aballaby nbsp',
 'abandon',
 'abandon days',
 'abandon every',
 'abandon may',
 'abandoned',
 'abandoned although',
 'abandoned area',
 'abandoning',
 'abandoning isreal',
 'abandons',
 'abandons hate',
 'abated',
 'abated came',
 'abba',
 'abba gold',
 'abba tune',
 'abc',
 'abc last',
 'abc nightline',
 'abcfamily',
 'abcfamily tapes',
 'abcnews',
 'abcnews com',
 'abcnews go',
 'abdicate',
 'abdicate ove

#### View term-document matrix

In [75]:
X_train_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [76]:
X_train_bow.shape

(2400, 180703)

### Create a dictionary to get label counts

In [77]:
label_counts = dict()

for labels in df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

#### Print the dictionary

In [78]:
label_counts

{'male': 2272,
 '15': 299,
 'Student': 403,
 'Leo': 55,
 '33': 94,
 'InvestmentBanking': 70,
 'Aquarius': 286,
 'female': 728,
 '14': 74,
 'indUnk': 452,
 'Aries': 1699,
 '25': 110,
 'Capricorn': 77,
 '17': 147,
 'Gemini': 21,
 '23': 93,
 'Non-Profit': 46,
 'Cancer': 76,
 'Banking': 16,
 '37': 19,
 'Sagittarius': 113,
 '26': 43,
 '24': 334,
 'Scorpio': 243,
 '27': 86,
 'Education': 118,
 '45': 14,
 'Engineering': 119,
 'Libra': 313,
 'Science': 33,
 '34': 6,
 '41': 14,
 'Communications-Media': 14,
 'BusinessServices': 21,
 'Sports-Recreation': 75,
 'Virgo': 39,
 'Taurus': 76,
 'Arts': 2,
 'Pisces': 2,
 '44': 3,
 '16': 25,
 'Internet': 20,
 'Museums-Libraries': 2,
 'Accounting': 2,
 '39': 32,
 '35': 1607,
 'Technology': 1607}

## Multi label binarizer

Load a multilabel binarizer and fit it on the labels.

In [79]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=sorted(label_counts.keys()))
y_train = mlb.fit_transform(y_train)
y_test = mlb.transform(y_test)

In [80]:
y_test

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 1, 1, 0]])

In [81]:
y_test.shape

(600, 47)

In [82]:
X_train_bow

<2400x180703 sparse matrix of type '<class 'numpy.int64'>'
	with 326981 stored elements in Compressed Sparse Row format>

## Classifier

Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on every label.

In [83]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs')
clf = OneVsRestClassifier(clf)

### Fit the classifier

In [84]:
clf.fit(X_train_bow, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [93]:
desired_vocab_size = 10000 #Vocablury size
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size) # num_words -> Vocablury size

In [95]:
#Fit tokenizer with actual training data
t.fit_on_texts(X_train.tolist())

In [96]:
#Vocabulary
t.word_index

{'like': 1,
 'one': 2,
 'get': 3,
 'know': 4,
 'go': 5,
 'think': 6,
 'would': 7,
 'time': 8,
 'really': 9,
 'urllink': 10,
 'good': 11,
 'well': 12,
 'people': 13,
 'nbsp': 14,
 'got': 15,
 'back': 16,
 'much': 17,
 'see': 18,
 'going': 19,
 'last': 20,
 'day': 21,
 'could': 22,
 'even': 23,
 'want': 24,
 'way': 25,
 'love': 26,
 'said': 27,
 'say': 28,
 'something': 29,
 'still': 30,
 'today': 31,
 'new': 32,
 'right': 33,
 'little': 34,
 'went': 35,
 'make': 36,
 'things': 37,
 'first': 38,
 'never': 39,
 'us': 40,
 'thing': 41,
 'work': 42,
 'life': 43,
 'also': 44,
 'need': 45,
 'night': 46,
 'take': 47,
 'around': 48,
 'home': 49,
 'feel': 50,
 'though': 51,
 'another': 52,
 'world': 53,
 'many': 54,
 'actually': 55,
 'pretty': 56,
 'next': 57,
 'year': 58,
 'thought': 59,
 'oh': 60,
 'friends': 61,
 'ever': 62,
 'come': 63,
 'two': 64,
 'u': 65,
 'long': 66,
 'made': 67,
 'look': 68,
 'maybe': 69,
 'let': 70,
 'always': 71,
 'since': 72,
 'place': 73,
 'guess': 74,
 'week': 75,


In [97]:
X_train

array(['begin today little update oral skank still gotten back get response someone else tell think excellent utils including dsl cable right parking etc sounds good well may screen name involves numbers little concerned maybe juvenile enough think number funny something grow like th grade well mailed back asked questions move would mid feb map looks really easy get east okay new subject sister brought last night jeopardy say form question really expound like got jeopardy instead saying jupiter say jupiter went back forth got little hand possibilities jeopardy answers old way american bulldog milan elizabeth taylor top old smokey pizza new way run american bulldog whorehouses milan anybody really care elizabeth taylor ever sex top old smokey throw eating much pizza see improvement stimu lator',
       'ok see tagbord leave messages storta stuff leave message people',
       'times life felt helpless perhaps acute pain person know founded frustration andand ventless rage nick sword upon

In [98]:
type(X_train[3671112])

IndexError: index 3671112 is out of bounds for axis 0 with size 2400

In [99]:
#Replace each word in the text with word's index
X_train = t.texts_to_sequences(X_train.tolist())

In [100]:
X_train

[[994,
  31,
  34,
  938,
  8588,
  5545,
  30,
  885,
  16,
  3,
  1333,
  79,
  144,
  101,
  6,
  2580,
  919,
  3029,
  1855,
  33,
  1856,
  465,
  472,
  11,
  12,
  106,
  1194,
  113,
  1443,
  34,
  1857,
  69,
  95,
  6,
  466,
  185,
  29,
  1334,
  1,
  208,
  995,
  12,
  6663,
  16,
  202,
  455,
  351,
  7,
  1858,
  6664,
  2395,
  352,
  9,
  473,
  3,
  939,
  214,
  32,
  886,
  359,
  531,
  20,
  46,
  6665,
  28,
  768,
  322,
  9,
  6666,
  1,
  15,
  6665,
  362,
  274,
  8589,
  28,
  8589,
  35,
  16,
  3328,
  15,
  34,
  311,
  3329,
  6665,
  1384,
  89,
  25,
  492,
  8590,
  8591,
  6667,
  5546,
  251,
  89,
  5547,
  1782,
  32,
  25,
  193,
  492,
  8590,
  8591,
  965,
  9,
  205,
  6667,
  5546,
  62,
  790,
  251,
  89,
  5547,
  1056,
  474,
  17,
  1782,
  18,
  4719,
  4720],
 [148, 18, 218, 2235, 114, 218, 996, 13],
 [122,
  43,
  215,
  5548,
  425,
  8592,
  415,
  110,
  4,
  4126,
  4721,
  1962,
  2396,
  4127,
  569,
  4722,
  4128,
  2236

In [101]:
len(X_train[1])

8

In [102]:
X_test = t.texts_to_sequences(X_test)

In [103]:
print(X_train[0])

[994, 31, 34, 938, 8588, 5545, 30, 885, 16, 3, 1333, 79, 144, 101, 6, 2580, 919, 3029, 1855, 33, 1856, 465, 472, 11, 12, 106, 1194, 113, 1443, 34, 1857, 69, 95, 6, 466, 185, 29, 1334, 1, 208, 995, 12, 6663, 16, 202, 455, 351, 7, 1858, 6664, 2395, 352, 9, 473, 3, 939, 214, 32, 886, 359, 531, 20, 46, 6665, 28, 768, 322, 9, 6666, 1, 15, 6665, 362, 274, 8589, 28, 8589, 35, 16, 3328, 15, 34, 311, 3329, 6665, 1384, 89, 25, 492, 8590, 8591, 6667, 5546, 251, 89, 5547, 1782, 32, 25, 193, 492, 8590, 8591, 965, 9, 205, 6667, 5546, 62, 790, 251, 89, 5547, 1056, 474, 17, 1782, 18, 4719, 4720]


#### Pad Sequences

In [104]:
#Define maximum number of words to consider in each blog, lets take it average = 289
max_blog_length = 289

In [105]:
#Pad training and test blogs
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        maxlen=max_blog_length,
                                                        padding='post',
                                                        truncating='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, 
                                                       maxlen=max_blog_length, 
                                                       padding='post',
                                                       truncating='post')

In [106]:
X_train.shape

(2400, 289)

In [107]:
X_test.shape

(600, 289)

In [108]:
X_train[23]

array([  26,  254, 2594, 2410, 8632, 1246,  664,  157,   98,  193,  270,
        445,  368, 1515,  297, 1149,  572, 8633, 4747,  861,  445,   44,
       4148,  420, 5573,    6,  119, 6705,   26, 4149,    9,  549, 6706,
       1149, 2411, 4748, 5574,   17,   44,   26, 1870, 1388, 8634, 1967,
       6707, 1516, 4150,  264,  152, 8635, 1450,   26,   87, 8636, 4749,
         44,    9,  602, 4748,  119,  752,  696, 6708,   90,    6,   33,
        943,  297,   25,   17, 1702,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

#### Load Google Word2Vec model

We can use gensim library to load pre-trained Word2Vec or Glove models. For list of available models can be found at [this url](https://github.com/RaRe-Technologies/gensim-data).

In [109]:
#Load Google word2vec model
model = api.load('word2vec-google-news-300')

In [110]:
#Size of the model
print(model.vectors.shape)

(3000000, 300)


In [111]:
print(model.vector_size)

300


#### Get Pre-trained Embeddings

In [112]:
embedding_vector_length = model.vector_size

Google Word2Vec model has vocabulary size of 3M words. In this example, we have only 10000 words as vocabulary. This means we do not require entire Google Word2Vec model. Rather, we will only take the embeddings of the words that are in our dataset vocabulary.

In [113]:
#Initialize embedding matrix for our dataset with 10000+1 rows (1 for padding word)
#and 300 columns (as embedding size is 300)
embedding_matrix = np.zeros((desired_vocab_size + 1, embedding_vector_length))

In [114]:
#Load word vectors for each word from Google Word2Vec model
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (desired_vocab_size+1):
        break
    try:
        embedding_vector = model[word] #Reading word's embedding from Google Word2Vec
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [115]:
embedding_matrix.shape

(10001, 300)

#### Build Model - Dense Layers

In [116]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [117]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    embedding_vector_length, #Embedding size
                                    weights=[embedding_matrix], #Embeddings taken from pre-trained model
                                    trainable=False, #As embeddings are already available, we will not train this layer. It will act as lookup layer.
                                    input_length=max_blog_length) #Number of words in each review
          )

Embedding Layer gives us 3D output ->
[Batch_Size , Review Length , Embedding_Size]

In [118]:
model.output

<tf.Tensor 'embedding/embedding_lookup/Identity_1:0' shape=(None, 289, 300) dtype=float32>

In [119]:
#Flatten the data as we will use Dense layers
model.add(tf.keras.layers.Flatten())

In [120]:
model.output

<tf.Tensor 'flatten/Reshape:0' shape=(None, 86700) dtype=float32>

Add Hidden layers

In [121]:
#Add Hidden layers (Dense layers)
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

Add Output layer

In [126]:
model.add(tf.keras.layers.Dense(47, activation='sigmoid'))

In [127]:
#Compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [128]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 289, 300)          3000300   
_________________________________________________________________
flatten (Flatten)            (None, 86700)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               8670100   
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1

##### Train Model

In [129]:
history = model.fit(X_train,y_train,
          epochs=200,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/200
75/75 [==============================] - 2s 29ms/step - loss: 12.9085 - accuracy: 0.0012 - val_loss: 11.2271 - val_accuracy: 0.0000e+00
Epoch 2/200
75/75 [==============================] - 2s 25ms/step - loss: 10.5745 - accuracy: 0.0000e+00 - val_loss: 10.7084 - val_accuracy: 0.0000e+00
Epoch 3/200
75/75 [==============================] - 2s 25ms/step - loss: 10.0888 - accuracy: 0.0000e+00 - val_loss: 10.4752 - val_accuracy: 0.0000e+00
Epoch 4/200
75/75 [==============================] - 2s 25ms/step - loss: 9.6681 - accuracy: 0.0000e+00 - val_loss: 10.7485 - val_accuracy: 0.0000e+00
Epoch 5/200
75/75 [==============================] - 2s 25ms/step - loss: 9.3863 - accuracy: 0.0000e+00 - val_loss: 10.8269 - val_accuracy: 0.0000e+00
Epoch 6/200
75/75 [==============================] - 2s 25ms/step - loss: 9.2011 - accuracy: 0.0000e+00 - val_loss: 10.9008 - val_accuracy: 0.0000e+00
Epoch 7/200
75/75 [==============================] - 2s 26ms/step - loss: 9.0884 - accuracy: 0.

75/75 [==============================] - 2s 26ms/step - loss: 7.3123 - accuracy: 0.1229 - val_loss: 12.6251 - val_accuracy: 0.0067
Epoch 58/200
75/75 [==============================] - 2s 25ms/step - loss: 7.3118 - accuracy: 0.1088 - val_loss: 12.9869 - val_accuracy: 0.0767
Epoch 59/200
75/75 [==============================] - 2s 26ms/step - loss: 7.3462 - accuracy: 0.1092 - val_loss: 12.8405 - val_accuracy: 0.0017
Epoch 60/200
75/75 [==============================] - 2s 27ms/step - loss: 7.3163 - accuracy: 0.1417 - val_loss: 12.9800 - val_accuracy: 0.0000e+00
Epoch 61/200
75/75 [==============================] - 2s 25ms/step - loss: 7.2646 - accuracy: 0.1225 - val_loss: 12.9191 - val_accuracy: 0.0050
Epoch 62/200
75/75 [==============================] - 2s 24ms/step - loss: 7.2639 - accuracy: 0.1142 - val_loss: 12.8701 - val_accuracy: 0.0000e+00
Epoch 63/200
75/75 [==============================] - 2s 24ms/step - loss: 7.2223 - accuracy: 0.1538 - val_loss: 12.9093 - val_accuracy: 0.01

Epoch 114/200
75/75 [==============================] - 2s 31ms/step - loss: 6.9522 - accuracy: 0.1400 - val_loss: 13.5472 - val_accuracy: 0.0067
Epoch 115/200
75/75 [==============================] - 2s 30ms/step - loss: 6.8834 - accuracy: 0.1942 - val_loss: 13.6974 - val_accuracy: 0.4300
Epoch 116/200
75/75 [==============================] - 2s 28ms/step - loss: 6.8937 - accuracy: 0.1292 - val_loss: 13.8183 - val_accuracy: 0.2983
Epoch 117/200
75/75 [==============================] - 2s 28ms/step - loss: 6.9372 - accuracy: 0.1238 - val_loss: 13.9051 - val_accuracy: 0.0050
Epoch 118/200
75/75 [==============================] - 2s 27ms/step - loss: 6.8775 - accuracy: 0.1521 - val_loss: 13.7561 - val_accuracy: 0.0783
Epoch 119/200
75/75 [==============================] - 2s 29ms/step - loss: 6.8771 - accuracy: 0.0983 - val_loss: 13.3438 - val_accuracy: 0.0017
Epoch 120/200
75/75 [==============================] - 2s 30ms/step - loss: 6.8691 - accuracy: 0.1600 - val_loss: 13.6532 - val_ac

75/75 [==============================] - 2s 27ms/step - loss: 6.7628 - accuracy: 0.1404 - val_loss: 14.1363 - val_accuracy: 0.3333
Epoch 171/200
75/75 [==============================] - 2s 27ms/step - loss: 6.6904 - accuracy: 0.1267 - val_loss: 14.1203 - val_accuracy: 0.0200
Epoch 172/200
75/75 [==============================] - 2s 27ms/step - loss: 6.7099 - accuracy: 0.1379 - val_loss: 13.7562 - val_accuracy: 0.0000e+00
Epoch 173/200
75/75 [==============================] - 2s 27ms/step - loss: 6.7544 - accuracy: 0.1488 - val_loss: 13.9729 - val_accuracy: 0.0033
Epoch 174/200
75/75 [==============================] - 2s 28ms/step - loss: 6.7272 - accuracy: 0.1146 - val_loss: 13.9403 - val_accuracy: 0.0067
Epoch 175/200
75/75 [==============================] - 2s 27ms/step - loss: 6.7372 - accuracy: 0.1408 - val_loss: 13.8562 - val_accuracy: 0.0017
Epoch 176/200
75/75 [==============================] - 2s 27ms/step - loss: 6.7885 - accuracy: 0.1717 - val_loss: 13.9547 - val_accuracy: 0.